In [1]:
import os
from tree_utils import Tree

In [2]:
data_dir = '/Users/xy/data/PennTreebank_full/parsed/prd/wsj'
assert os.path.exists(data_dir), 'data_dir not found'

In [3]:
# How many subdirectories
sub_dirs = [sd for sd in os.listdir(data_dir) \
            if os.path.isdir(os.path.join(data_dir, sd))]
sub_dirs = sorted(sub_dirs)
print(f'Found {len(sub_dirs)} subdirectories in {data_dir}')
print(sub_dirs)

Found 25 subdirectories in /Users/xy/data/PennTreebank_full/parsed/prd/wsj
['00', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24']


In [4]:
# How many .prd files exist in total
prd_files = [os.path.join(data_dir, sd, f) \
             for sd in sub_dirs \
                for f in os.listdir(os.path.join(data_dir, sd)) \
                    if f.endswith('.prd')]
print(f'Found {len(prd_files)} .prd files in total')
print('Examples:', prd_files[:1])

Found 2312 .prd files in total
Examples: ['/Users/xy/data/PennTreebank_full/parsed/prd/wsj/00/wsj_0042.prd']


In [5]:
# Work on a .prd example
example_prd_file = '/Users/xy/data/PennTreebank_full/parsed/prd/wsj/00/wsj_0008.prd'

tree_strings = []
with open(example_prd_file) as f:
    lp_count, rp_count = 0, 0
    tmp_str = ''
    for line in f:
        line = line.strip()
        if not line: continue
        lp_count += line.count('(')
        rp_count += line.count(')')
        if lp_count > rp_count:
            tmp_str += ' ' + line
        elif lp_count == rp_count:
            tmp_str += ' ' + line
            tree_strings.append(tmp_str)
            tmp_str = ''
            lp_count, rp_count = 0, 0
        else:
            raise ValueError('Something wrong with the parentheses')

print(f'Found {len(tree_strings)} trees in {example_prd_file}')
print('Examples:')
print(tree_strings[0])

Found 6 trees in /Users/xy/data/PennTreebank_full/parsed/prd/wsj/00/wsj_0008.prd
Examples:
 ( (S (NP-SBJ The federal government) (VP suspended (NP (NP sales) (PP of (NP U.S. savings bonds))) (SBAR-PRP because (S (NP-SBJ Congress) (VP has n't (VP lifted (NP (NP the ceiling) (PP-LOC on (NP government debt)))))))) .))


In [6]:
# Wrap up the above ops into a function
def get_tree_strings(prd_file):
    tree_strings = []
    with open(prd_file) as f:
        lp_count, rp_count = 0, 0
        tmp_str = ''
        for line in f:
            line = line.strip()
            if not line: continue
            lp_count += line.count('(')
            rp_count += line.count(')')
            if lp_count > rp_count:
                tmp_str += ' ' + line # add a space to separate words/phrasal structures
            elif lp_count == rp_count:
                tmp_str += ' ' + line # same as above
                tree_strings.append(tmp_str.strip())
                tmp_str = ''
                lp_count, rp_count = 0, 0
            else:
                raise ValueError('Something wrong with the parentheses')
    return tree_strings

# Test the function
tree_strings = get_tree_strings(example_prd_file)
print(f'Found {len(tree_strings)} trees in {example_prd_file}')
print('Examples:')
print(tree_strings[0])

Found 6 trees in /Users/xy/data/PennTreebank_full/parsed/prd/wsj/00/wsj_0008.prd
Examples:
( (S (NP-SBJ The federal government) (VP suspended (NP (NP sales) (PP of (NP U.S. savings bonds))) (SBAR-PRP because (S (NP-SBJ Congress) (VP has n't (VP lifted (NP (NP the ceiling) (PP-LOC on (NP government debt)))))))) .))


In [7]:
tree0 = Tree(tree_strings[0])

print(tree0)
print(tree0.leaves())

tree1 = Tree(tree_strings[1])
print(tree1)
print(tree1.leaves())

# sample a random tree string
import random
print('Sample a random tree string:')
random_tree_string = random.choice(tree_strings)
print(random_tree_string)
random_tree = Tree(random_tree_string)
print(random_tree.leaves())

(S ('NP-SBJ', ['The federal government'])('VP', ['suspended', ('NP', [('NP', ['sales']), ('PP', ['of', ('NP', ['U.S. savings bonds'])])]), ('SBAR-PRP', ['because', ('S', [('NP-SBJ', ['Congress']), ('VP', ["has n't", ('VP', ['lifted', ('NP', [('NP', ['the ceiling']), ('PP-LOC', ['on', ('NP', ['government debt'])])])])])])])]).)
['The federal government', 'suspended', 'sales', 'of', 'U.S. savings bonds', 'because', 'Congress', "has n't", 'lifted', 'the ceiling', 'on', 'government debt', '.']
(S ('S-TPC-1', [('SBAR-TMP', ['Until', ('S', [('NP-SBJ', ['Congress']), ('VP', ['acts'])])]), ',', ('NP-SBJ', ['the government']), ('VP', ["has n't", ('NP', ['any authority', ('S', [('NP-SBJ', ['*']), ('VP', ['to', ('VP', ['issue', ('NP', [('NP', ['new debt obligations']), ('PP', ['of', ('NP', ['any kind'])])])])])])])])]),('NP-SBJ', ['the Treasury'])('VP', ['said', ('SBAR', ['0', ('S', ['*T*-1'])])]).)
['Until', 'Congress', 'acts', ',', 'the government', "has n't", 'any authority', '*', 'to', 'issue

In [8]:
# Test over all prd files
for prd_file in prd_files:
    tree_strings = get_tree_strings(prd_file)
    for tree_str in tree_strings:
        try:
            tree = Tree(tree_str)
        except Exception as e:
            print('Parsing problem with tree string:')
            print(f'prd_file: {prd_file}')
            print(f'tree_str: {tree_str}')
            print(e)
            raise

In [9]:
# extract all leaves and save to text files
import os
from tqdm import tqdm

output_dir = '/Users/xy/data/PennTreebank_full/prd_leaves/wsj'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

for dir in sub_dirs: # create subdirectories
    dir_path = os.path.join(output_dir, dir)
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

# Read all .prd files and extract leaves
for prd_file in tqdm(prd_files):
    tree_strings = get_tree_strings(prd_file)
    leaf_strings = []
    for tree_str in tree_strings:
        tree = Tree(tree_str)
        leaves = tree.leaves()
        leaf_strings.append(' '.join(leaves))

    sub_dir = os.path.basename(os.path.dirname(prd_file)) # ../00/wsj_0001.prd -> 00
    filename = os.path.splitext(os.path.basename(prd_file))[0] # wsj_0001.prd -> wsj_0001
    output_file = os.path.join(output_dir, sub_dir, filename + '.txt')
    with open(output_file, 'w') as f:
        f.write('\n'.join(leaf_strings))

100%|██████████| 2312/2312 [00:03<00:00, 706.69it/s]
